# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())


['establishments']


In [6]:
# review the collections in our new database
collection_names = db.list_collection_names()
print(collection_names)


['establishments']


In [7]:
# review a document in the establishments collection
document = db.establishments.find_one()
pprint(document)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6637b4c36e1b30be51ce3f54'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [10]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [12]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "Name": "Penang Flavours",
    "AddressLine1": "123 Greenwich Street",
    "AddressLine2": "",
    "AddressLine3": "",
    "PostCode": "SE10 8JA",
    "RatingValue": None,  
    "HygieneScore": None,  
    "LocalAuthorityName": "Greenwich",
    "Latitude": 51.4805,  
    "Longitude": -0.0044,  
}


In [15]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [16]:
print("Inserted document ID:", insert_result.inserted_id)

Inserted document ID: 6637c38651f91862a4f77ac7


In [18]:
# Check that the new restaurant was inserted
inserted_restaurant = establishments.find_one({"Name": "Penang Flavours"})

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [19]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Project only the BusinessTypeID and BusinessType fields
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the specified fields
business_type = db.business_types.find_one(query, projection)

# Print the result
if business_type:
    pprint(business_type)
else:
    print("Business type not found.")

Business type not found.


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
filter = {"Name": "Penang Flavours"}

# Define the update operation to set the BusinessTypeID
update_operation = {
    "$set": {"BusinessTypeID": business_type["BusinessTypeID"]}
}

# Update the new restaurant with the correct BusinessTypeID
update_result = establishments_collection.update_one(filter, update_operation)

# Print the update result
print("Matched:", update_result.matched_count)
print("Modified:", update_result.modified_count)

In [ ]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"Name": "Penang Flavours"})

# Print the details of the updated restaurant
if updated_restaurant:
    print("Updated restaurant details:")
    pprint(updated_restaurant)
else:
    print("The updated restaurant was not found in the collection.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

count = establishments.count_documents(query)

print("Number of documents with LocalAuthorityName as Dover:", count)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many(query)


In [ ]:
# Check if any remaining documents include Dover
remaining_count = establishments.count_documents(query)


In [ ]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

# Print
if remaining_document:
    print("Other documents remain in the collection.")
    pprint(remaining_document)
else:
    print("No other documents found in the collection.")

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
from bson.decimal128 import Decimal128

filter = {"Latitude": {"$type": "string"}, "Longitude": {"$type": "string"}}

# Define the update operation to convert latitude and longitude to decimals
update_operation = {
    "$set": {
        "Latitude": {"$toDouble": "$Latitude"},
        "Longitude": {"$toDouble": "$Longitude"}
    }
}

# Update the data type for latitude and longitude fields
update_result = establishments_collection.update_many(filter, update_operation)

print("Number of documents updated:", update_result.modified_count)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
update_result = establishments.update_many(
    {"RatingValue": {"$type": "string"}},  
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  
)


In [21]:
# Check that the coordinates and rating value are now numbers
document = establishments.find_one()  

if document:
    print("Details of one document from the collection:")
    pprint(document)
else:
    print("No documents found in the collection.")

Details of one document from the collection:
{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6637b4c36e1b30be51ce3f54'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource':